In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
%matplotlib inline
torch.cuda.empty_cache()

# Define Dataloaders for each class


In [2]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

############################################define batch size for all data loaders here
bathsize=32

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor()
    ])

labels=["No_Beard","Male","Bald","Bangs","Smiling"]
label_number=1
#Choose which label 
ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
dataset_loader=DataLoader(ds,batch_size=bathsize, shuffle=True, num_workers=workers)

ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)

"""
# male
ds1=Binary_Dataset(labels_dir, labels[1],image_dir ,transform=composed)
dataset_loader=DataLoader(ds1,batch_size=bathsize, shuffle=True, num_workers=workers)

# bald
ds2=Binary_Dataset(labels_dir, labels[2],image_dir ,transform=composed)
dataset_loader=DataLoader(ds2,batch_size=bathsize, shuffle=True, num_workers=workers)

# bangs
ds3=Binary_Dataset(labels_dir, labels[3],image_dir ,transform=composed)
dataset_loader=DataLoader(ds3,batch_size=bathsize, shuffle=True, num_workers=workers)

# smiling
ds4=Binary_Dataset(labels_dir, labels[4],image_dir ,transform=composed)
dataset_loader=DataLoader(ds4,batch_size=bathsize, shuffle=True, num_workers=workers)
"""

"""
for idx, (data, image) in enumerate(ds):
    print(idx)
"""


data=next(iter(dataset_loader))
sample, target = data
#print(sample)
print(target)


tensor([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0])


In [3]:
"""
#wont work anymore because pil
idx=0
plt.imshow(sample[idx])
print(labels[label_number])
print(target[idx])
"""

'\n#wont work anymore because pil\nidx=0\nplt.imshow(sample[idx])\nprint(labels[label_number])\nprint(target[idx])\n'

# Define CNN's

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3(pretrained=False).to(device)

# Training Loop

In [5]:
learning_rate=0.0001
optimizer = Adam(params=model.parameters(), lr=learning_rate)
train_loop(model=model,dataloader=dataset_loader,epochs=1,optimizer=optimizer,save_name="Male_12_8")

tensor(11.5112, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.3254, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.9721, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.3766, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.3308, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.6469, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.9748, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.4418, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.7721, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.2523, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.9391, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.4409, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.8027, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12.5201, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.2041, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.0777, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.8292, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.3235, device='cuda:0', gra

KeyboardInterrupt: 

In [12]:
#check validation accuracy
ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)
device = torch.device("cpu")
val_data=iter(val_loader)
model=model.to(device)
correct=0
tested_samples=0
limit=2000
for data in val_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)

    pred=torch.argmax(model(sample),dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)

tensor(25)
tensor(47)
tensor(67)
tensor(87)
tensor(107)
tensor(129)
tensor(152)
tensor(173)
tensor(190)
tensor(213)
tensor(234)
tensor(261)
tensor(281)
tensor(308)
tensor(331)
tensor(350)
tensor(371)
tensor(397)
tensor(415)
tensor(439)
tensor(458)
tensor(475)
tensor(496)
tensor(517)
tensor(540)
tensor(564)
tensor(586)
tensor(609)
tensor(630)
tensor(649)
tensor(672)
0.03503296924548246


0.6796875
